<a href="https://colab.research.google.com/github/arohanajit/Coursera_Capstone/blob/master/Toronto_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import os
import numpy as np
import pandas as pd
import json
import geocoder 
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


### Extracting Data
* URL is taken through requests method
* Using Beautiful soup library, table data is extracted

In [2]:
pageUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(pageUrl).text

In [3]:
soup = BeautifulSoup(page, 'html.parser')
myTable = soup.find('table', class_="wikitable sortable")
headings = myTable.find_all('th')

### Pandas DataFrame
* An empty dataframe is created using extracted headings
* Values are extracted from table and appended after formatting
  * Not assigned values are removed first
  * White spaces are removed from list
  * Boroughs with multiple neighbourhoods are formatted

In [4]:
column = []
for i in headings:
    column.append(i.text[:-1])
toronto_hood = pd.DataFrame(columns=column)
toronto_hood

,Postal code,Borough,Neighborhood


In [5]:
values = myTable.find_all('tr')
del values[0]
for i in values:
    temp = i.text.split('\n')
    if temp[3]=='Not assigned':
        continue
    else:
        temp = [temp[i] for i in range(len(temp)) if i%2!=0]
        temp[-1] = temp[-1].replace(' /',',')
        toronto_hood = toronto_hood.append({'Postal code':temp[0],
                                            'Borough':temp[1],
                                            'Neighborhood':temp[2]},ignore_index=True)

        

In [6]:
toronto_hood = toronto_hood.groupby(['Postal code','Borough'],as_index=False).agg(lambda x: ", ".join(x))
for index, row in toronto_hood.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
toronto_hood.head(10)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
toronto_hood.shape

(103, 3)

In [8]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
coordinates.rename(columns={"Postal Code": "Postal code"}, inplace=True)
coordinates.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [124]:
toronto_new = toronto_hood.merge(coordinates, on="Postal code", how="left")
toronto_new.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
print("Toronto has {} Boroughs and {} unique neighborhoods".format(len(toronto_new['Borough'].unique()),
                                                                       toronto_new.shape[0]))

Toronto has 10 Boroughs and 103 unique neighborhoods


In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(toronto_new['Latitude'], toronto_new['Longitude'], toronto_new['Borough'], toronto_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [125]:
highlight_hood = [i for i in toronto_new['Borough'].unique() if 'Toronto' in i]
print(highlight_hood)
subtoronto_data = toronto_new[toronto_new['Borough'].isin(highlight_hood)].reset_index(drop=True)
print(subtoronto_data.shape)
subtoronto_data.head()

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
(39, 5)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [127]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(subtoronto_data['Latitude'], subtoronto_data['Longitude'], 
                                           subtoronto_data['Borough'], subtoronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [128]:
CLIENT_ID = '3KHJVC5U5PAOGTOZXR2Y5KCYDNGLUYOYDMY5LILY2W34MCAO' 
CLIENT_SECRET = 'L0O2VAQ2ETKJJPGWJ554DHC3052O2AVWGCTNYXEGFDG10D11' 
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: 3KHJVC5U5PAOGTOZXR2Y5KCYDNGLUYOYDMY5LILY2W34MCAO
CLIENT_SECRET:L0O2VAQ2ETKJJPGWJ554DHC3052O2AVWGCTNYXEGFDG10D11


In [129]:
LIMIT = 100
radius = 500
count = 0
venues_list = []
for lat,long,name,bor,code in zip(subtoronto_data['Latitude'],subtoronto_data['Longitude'],
                                  subtoronto_data['Neighborhood'],subtoronto_data['Borough'],
                                  subtoronto_data['Postal code']):
    

    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    if count%5==0:
        print("{} done".format(count))
    tries=0
    while(1):
        tries+=1
        if tries%20==0:
            print("Try : {}...".format(tries))
        GET = requests.get(url).json()["response"]
        if len(GET)!=0:
            break
    results = GET['groups'][0]['items']
    count+=1
    venues_list.append([(
            code,
            bor,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

0 done
5 done
10 done
15 done
20 done
25 done
30 done
35 done


In [137]:
for i in venues_list:
    for j in range(len(i)):
        if i[j][-1]=='Neighborhood':
            i[j] = tuple(list(i[j][:-1]) + ['Community'])

In [138]:
venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
venues.columns = [
                'Postal code',
                'Borough',
                'Neighborhood', 
                'Borough Latitude', 
                'Borough Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']

In [139]:
print(venues.shape)
venues.head()

(1612, 9)


,Postal code,Borough,Neighborhood,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,East Toronto,The Beaches,43.676357,-79.321558,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.321558,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.321558,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.321558,Upper Beaches,43.680563,-79.292869,Community
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.321558,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [140]:
venues.groupby(["Postal code", "Borough", "Neighborhood"]).count()

,,,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal code,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",43,43,43,43,43,43
M4L,East Toronto,"India Bazaar, The Beaches West",18,18,18,18,18,18
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,6,6,6,6,6,6
M4R,Central Toronto,North Toronto West,21,21,21,21,21,21
M4S,Central Toronto,Davisville,33,33,33,33,33,33
M4T,Central Toronto,"Moore Park, Summerhill East",2,2,2,2,2,2


In [141]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 231 uniques categories.


In [142]:
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Postal code'] = venues['Postal code'] 
toronto_onehot['Borough'] = venues['Borough'] 
toronto_onehot['Neighborhood'] = venues['Neighborhood']
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Postal code,Borough,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
toronto_onehot.shape

(1612, 234)

In [144]:
toronto_grouped = toronto_onehot.groupby(["Postal code", "Borough", "Neighborhood"]).mean().reset_index()
toronto_grouped

,Postal code,Borough,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,...,0.000000,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.025000
4,M4N,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619
7,M4S,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,...,0.000000,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000


In [145]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [146]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postal code', 'Borough', 'Neighborhood']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal code'] = toronto_grouped['Postal code']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.head()

,Postal code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Trail,Community,Pub,Health Food Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Café,Spa,Japanese Restaurant,Brewery
2,M4L,East Toronto,"India Bazaar, The Beaches West",Fast Food Restaurant,Sandwich Place,Gym,Pub,Fish & Chips Shop,Sushi Restaurant,Brewery,Ice Cream Shop,Liquor Store,Pizza Place
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Brewery,Bakery,American Restaurant,Gastropub,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,Park,Dim Sum Restaurant,Bus Line,Swim School,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [147]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop(["Postal code", "Borough", "Neighborhood"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([0, 2, 2, 2, 0, 2, 2, 2, 3, 2])

In [148]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()
toronto_merged = subtoronto_data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(['Borough','Neighborhood'],1)
                                     .set_index('Postal code'), on='Postal code')
toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Community,Pub,Health Food Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Café,Spa,Japanese Restaurant,Brewery
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Fast Food Restaurant,Sandwich Place,Gym,Pub,Fish & Chips Shop,Sushi Restaurant,Brewery,Ice Cream Shop,Liquor Store,Pizza Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Brewery,Bakery,American Restaurant,Gastropub,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Dim Sum Restaurant,Bus Line,Swim School,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [149]:
print(toronto_merged.shape)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head()

(39, 16)


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Community,Pub,Health Food Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
23,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Dim Sum Restaurant,Bus Line,Swim School,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,2,Sandwich Place,Café,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Indian Restaurant,Flower Shop,BBQ Joint,History Museum,Liquor Store


In [150]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters